# Get relevant gene list.

### Read and manipulate gtf file

In [2]:
import pandas as pd
import re
import numpy as np
import json

In [3]:
PATH_GTF_FILE = "/homes/mcolombari/AI_for_Bioinformatics_Project/Personal/gencode.v47.annotation.gtf"
OUTPUT_FOLDER_GENE_ID = "."
SAVE_GENE_ID = True

In [4]:
gtf = pd.read_csv(PATH_GTF_FILE, sep="\t", header=None, comment='#')

In [5]:
gtf.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

parameters = ['gene_id', 'gene_type']
for p in parameters:
    gtf[p] = gtf['attribute'].apply(lambda x: re.findall(rf'{p} "([^"]*)"', x)[0] if rf'{p} "' in x else np.nan)

gtf.drop('attribute', axis=1, inplace=True)

print(gtf)

        seqname   source     feature  start    end score strand frame  \
0          chr1   HAVANA        gene  11121  24894     .      +     .   
1          chr1   HAVANA  transcript  11121  14413     .      +     .   
2          chr1   HAVANA        exon  11121  11211     .      +     .   
3          chr1   HAVANA        exon  12010  12227     .      +     .   
4          chr1   HAVANA        exon  12613  12721     .      +     .   
...         ...      ...         ...    ...    ...   ...    ...   ...   
4105480    chrM  ENSEMBL  transcript  15888  15953     .      +     .   
4105481    chrM  ENSEMBL        exon  15888  15953     .      +     .   
4105482    chrM  ENSEMBL        gene  15956  16023     .      -     .   
4105483    chrM  ENSEMBL  transcript  15956  16023     .      -     .   
4105484    chrM  ENSEMBL        exon  15956  16023     .      -     .   

                   gene_id gene_type  
0        ENSG00000290825.2    lncRNA  
1        ENSG00000290825.2    lncRNA  
2     

In [6]:
print(gtf.columns)

Index(['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand',
       'frame', 'gene_id', 'gene_type'],
      dtype='object')


In [7]:
gtf_pc = gtf[gtf['gene_type'] == 'protein_coding']
print(gtf_pc)

        seqname   source      feature  start    end score strand frame  \
2486       chr1   HAVANA         gene  65419  71585     .      +     .   
2487       chr1   HAVANA   transcript  65419  71585     .      +     .   
2488       chr1   HAVANA         exon  65419  65433     .      +     .   
2489       chr1   HAVANA         exon  65520  65573     .      +     .   
2490       chr1   HAVANA          CDS  65565  65573     .      +     0   
...         ...      ...          ...    ...    ...   ...    ...   ...   
4105474    chrM  ENSEMBL         gene  14747  15887     .      +     .   
4105475    chrM  ENSEMBL   transcript  14747  15887     .      +     .   
4105476    chrM  ENSEMBL         exon  14747  15887     .      +     .   
4105477    chrM  ENSEMBL          CDS  14747  15887     .      +     0   
4105478    chrM  ENSEMBL  start_codon  14747  14749     .      +     0   

                   gene_id       gene_type  
2486     ENSG00000186092.7  protein_coding  
2487     ENSG00000186

### Save Gene id relative to the protein coding

In [8]:
gtf_pc_set = set(gtf_pc['gene_id'].to_list())
print(len(gtf_pc_set))

20092


Value in output match with the stimated gene with are proteine coding.
source: [link](https://www.genome.gov/genetics-glossary/Gene#:~:text=And%20genes%20are%20the%20part,of%20the%20entire%20human%20genome.)

In [9]:
if SAVE_GENE_ID:
    with open(OUTPUT_FOLDER_GENE_ID + "/" + 'gene_id_protein_coding.json', 'w', encoding='utf-8') as f:
        json.dump(list(gtf_pc_set), f, ensure_ascii=False, indent=4) 

# Actual proprocessing

### Now parse load the data and parse it

In [10]:
import torch
import os

In [11]:
PATH_FOLDER_GENE = "/work/h2020deciderficarra_shared/TCGA/OV/project_n16_data/GeneExpression"
PATH_CASE_ID_STRUCTURE = "./case_id_and_structure.json"

In [12]:
with open(PATH_CASE_ID_STRUCTURE, 'r') as file:
    file_parsed = json.load(file)

In [13]:
file_to_case_id = dict((file_parsed[k]['files']['gene'], k) for k in file_parsed.keys())
file_to_os = dict((file_parsed[k]['files']['gene'], file_parsed[k]['os']) for k in file_parsed.keys())

Some resources to understand which feature to use:
- [Doc GDC](https://docs.gdc.cancer.gov/Data/Bioinformatics_Pipelines/Expression_mRNA_Pipeline/#introduction)

In [14]:
datastructure = pd.DataFrame(columns=['case_id', 'os', 'values'])

# All possibilitys.
feature_to_save = [
    'unstranded', 'stranded_first', 'stranded_second',
    'tpm_unstranded', 'fpkm_unstranded', 'fpkm_uq_unstranded'
    ]
# feature_to_save = ['unstranded']

index = 0
# Now explore data path to get the right files
for root, dirs, files in os.walk(PATH_FOLDER_GENE):
    for dir in dirs:
        for root, dirs, files in os.walk(PATH_FOLDER_GENE + "/" + dir):
            for file in files:
                if file in file_to_case_id.keys():
                    parsed_file = pd.read_csv(PATH_FOLDER_GENE + "/" + dir + "/" + file,
                                              sep='\t', header=0, skiprows=lambda x: x in [0, 2, 3, 4, 5])
                    parsed_file = parsed_file[['gene_id'] + feature_to_save]

                    # Now specify columns type.
                    convert_dict = dict([(k, float) for k in feature_to_save])
                    convert_dict['gene_id'] = str
                    parsed_file = parsed_file.astype(convert_dict)
                    
                    # They actually don't match.
                    # So the 'gene_type' in the dataset don't match the in the gtf file.
                    # So i'm gonna use as the right reference the gtf file.

                    # parsed_file = parsed_file[parsed_file['gene_type'] == 'protein_coding']
                    # if not set(parsed_file['gene_id']).issubset(gtf_pc_set):
                    #     raise Exception("List of coding genes don't match.")

                    parsed_file = parsed_file[parsed_file['gene_id'].isin(gtf_pc_set)]

                    datastructure.loc[index] = [
                        file_to_case_id[file],
                        file_to_os[file],
                        parsed_file
                    ]
                    index += 1

In [15]:
sizes = [datastructure.loc[i]['values'].shape[0] for i in range(datastructure.shape[0])]
print(f"Max: {max(sizes)}")
print(f"Min: {min(sizes)}")
print(f"Mean: {sum(sizes) / len(sizes)}")

Max: 14168
Min: 14168
Mean: 14168.0


### Apply data transformation

In [16]:
datastructure['values'].loc[0][feature_to_save]

,unstranded,stranded_first,stranded_second,tpm_unstranded,fpkm_unstranded,fpkm_uq_unstranded
1,2.0,2.0,0.0,0.0683,0.0246,0.0320
3,882.0,743.0,765.0,6.4589,2.3275,3.0306
4,427.0,572.0,522.0,3.6051,1.2991,1.6916
5,195.0,96.0,99.0,2.9062,1.0473,1.3636
7,1727.0,1281.0,1263.0,30.8464,11.1158,14.4736
...,...,...,...,...,...,...
60650,0.0,0.0,0.0,0.0000,0.0000,0.0000
60655,0.0,0.0,0.0,0.0000,0.0000,0.0000
60657,0.0,0.0,0.0,0.0000,0.0000,0.0000
60658,7.0,5.0,2.0,0.0367,0.0132,0.0172


In [17]:
for i in range(datastructure.shape[0]):
    datastructure['values'].loc[i][feature_to_save] = datastructure['values'].loc[i][feature_to_save].applymap(lambda x: np.log10(x + 0.01))

In [18]:
datastructure['values'].loc[0][feature_to_save]

,unstranded,stranded_first,stranded_second,tpm_unstranded,fpkm_unstranded,fpkm_uq_unstranded
1,0.303196,0.303196,-2.000000,-1.106238,-1.460924,-1.376751
3,2.945474,2.870995,2.883667,0.810830,0.368752,0.482959
4,2.630438,2.757404,2.717679,0.558120,0.116973,0.230857
5,2.290057,1.982316,1.995679,0.464817,0.024198,0.137860
7,3.237295,3.107553,3.101407,1.489345,1.046331,1.160877
...,...,...,...,...,...,...
60650,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000
60655,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000
60657,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000
60658,0.845718,0.699838,0.303196,-1.330683,-1.634512,-1.565431


For each row i normalize between 0 and 1.

In [19]:
for r in range(datastructure.shape[0]):
    for c in feature_to_save:
        datastructure['values'].loc[r][c] = (datastructure['values'].loc[r][c] - datastructure['values'].loc[r][c].min()) / \
                                            (datastructure['values'].loc[r][c].max() - datastructure['values'].loc[r][c].min())

In [20]:
# print(datastructure['values'].loc[0][feature_to_save])
for f in feature_to_save:
    print(f"{f}")
    print(f"\tMax: {float(datastructure['values'].loc[0][f].max())}")
    print(f"\tMin: {float(datastructure['values'].loc[0][f].min())}")

unstranded
	Max: 1.0
	Min: 0.0
stranded_first
	Max: 1.0
	Min: 0.0
stranded_second
	Max: 1.0
	Min: 0.0
tpm_unstranded
	Max: 1.0
	Min: 0.0
fpkm_unstranded
	Max: 1.0
	Min: 0.0
fpkm_uq_unstranded
	Max: 1.0
	Min: 0.0
